In [2]:
from sklearn.pipeline import make_pipeline
import numpy as np
from dense_subgraph import sdp, qp
import utils
from cs_classification import cs_p1_graphs_to_points, cs_p2_graphs_to_points
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin, BaseEstimator

In [3]:
class ContrastSubgraphTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, a_label, b_label,
                    alpha=None, alpha2=None,
                    percentile=70, percentile2=None,
                    problem=1, solver=sdp, num_cs=1) -> None:
        self.a_label = a_label
        self.b_label = b_label

        self.alpha = alpha
        if problem == 1:
            self.alpha2 = alpha2 or alpha
        
        self.alpha_is_provided = bool(self.alpha)

        self.percentile = percentile
        if problem == 1:
            self.percentile2 = percentile2 or percentile
        
        self.problem = problem
        self.solver = solver
        self.num_cs = num_cs

        # Lists for storing contrast subgraph(s)
        if problem == 1:
            self.cs_a_b_list = []
            self.cs_b_a_list = []
            self.find_cs = self.find_cs_p1
            self.axes_labels = [f"Number of edges overlapping with CS {a_label}-{b_label}",
                                f"Number of edges overlapping with CS {b_label}-{a_label}"]
        else: # problem == 2
            self.cs_list = []
            self.find_cs = self.find_cs_p2
            self.axes_labels = [r"L1 norm distance from $G^{%s}$"%a_label,
                                r"L1 norm distance from $G^{%s}$"%b_label]

    def fit(self, graphs, labels) -> None:
         # Create and Write Summary Graphs
        self.summary_A = utils.summary_graph(graphs[np.where(labels == self.a_label)])
        self.summary_B = utils.summary_graph(graphs[np.where(labels == self.b_label)])

        self.find_cs()

    def find_cs_p1(self) -> None:
        diff_a_b = self.summary_A - self.summary_B
        diff_b_a = self.summary_B - self.summary_A

        nodes = np.arange(diff_a_b.shape[0])
        node_mask_a_b = np.array([True]*nodes.shape[0])
        node_mask_b_a = np.array([True]*nodes.shape[0])

        for i in range(self.num_cs):
            masked_diff_a_b = utils.induce_subgraph(diff_a_b, nodes[node_mask_a_b])
            masked_diff_b_a = utils.induce_subgraph(diff_b_a, nodes[node_mask_b_a])

            # If no alpha value is provided, find the appropriate alpha value using the given percentile
            if not self.alpha_is_provided:
                # A -> B
                flat = masked_diff_a_b[np.triu_indices_from(masked_diff_a_b, k=1)]
                self.alpha = np.percentile(flat, self.percentile)
                print("alpha = {} ({}-th percentile)".format(self.alpha, self.percentile), end=", ")

                # B -> A
                flat = masked_diff_b_a[np.triu_indices_from(masked_diff_b_a, k=1)]
                self.alpha2 = np.percentile(flat, self.percentile2)
                print("alpha2 = {} ({}-th percentile)".format(self.alpha2, self.percentile2))

            cs_a_b = nodes[node_mask_a_b][self.solver(masked_diff_a_b, self.alpha)]
            self.cs_a_b_list.append(cs_a_b)
            cs_b_a = nodes[node_mask_b_a][self.solver(masked_diff_b_a, self.alpha2)]
            self.cs_b_a_list.append(cs_b_a)
            # Do not consider the previously found contrast subgraph nodes for future contrast subgraphs
            node_mask_a_b[cs_a_b] = False
            node_mask_b_a[cs_b_a] = False

            if len(nodes[node_mask_a_b]) == 0:
                print("Every node in the graph is included by a contrast subgraph(A->B)!\n\
                    Stopped at Contrast Subgraph {}.".format(i+1))
                break
            if len(nodes[node_mask_b_a]) == 0:
                print("Every node in the graph is included by a contrast subgraph (B->A)!\n\
                    Stopped at Contrast Subgraph {}.".format(i+1))
                break

    def find_cs_p2(self) -> None:
        diff = abs(self.summary_A - self.summary_B)

        nodes = np.arange(diff.shape[0])
        node_mask = np.array([True]*nodes.shape[0])
        
        for i in range(self.num_cs):
            masked_diff = utils.induce_subgraph(diff, nodes[node_mask])

            # If no alpha value is provided, find the appropriate alpha value using the given percentile
            if not self.alpha_is_provided:
                flat = masked_diff[np.triu_indices_from(masked_diff, k=1)]
                self.alpha = np.percentile(flat, self.percentile)
                print("alpha = {} ({}-th percentile)".format(self.alpha, self.percentile))
            
            cs = nodes[node_mask][self.solver(masked_diff, self.alpha)]
            self.cs_list.append(cs)
            node_mask[cs] = False

            if len(nodes[node_mask]) == 0:
                print("Every node in the graph is included by a contrast subgraph!\n\
                    Stopped at Contrast Subgraph {}.".format(i+1))
                break

    def transform(self, graphs):
        points = np.zeros((graphs.shape[0], 2))

        if self.problem == 1:
            num_cs = min(len(self.cs_a_b_list), len(self.cs_b_a_list))
            for i in range(num_cs):
                points += cs_p1_graphs_to_points(graphs=graphs,
                                                cs_a_b=self.cs_a_b_list[i],
                                                cs_b_a=self.cs_b_a_list[i])
        else: # problem == 2
            for i in range(len(self.cs_list)):
                points += cs_p2_graphs_to_points(graphs=graphs,
                                                contrast_subgraph=self.cs_list[i],
                                                summary_A=self.summary_A,
                                                summary_B=self.summary_B)
        return points

In [4]:
# Read brain graph files into numpy arrays
graphs_A = utils.get_graphs_from_files("./datasets/children/asd/")
graphs_B = utils.get_graphs_from_files("./datasets/children/td/")

graphs, labels = utils.label_and_concatenate_graphs(graphs_A, graphs_B, a_label="ASD", b_label="TD")

In [5]:
# Set up possible values of parameters to optimize over
p_grid = [{"svc__C": [0.1, 1, 10, 100],
          "svc__gamma": [0.00001,0.00005, 0.0001, 0.00015, 0.001, 0.01],
          "contrastsubgraphtransformer__alpha": [0.001, 0.01],
          "contrastsubgraphtransformer__alpha2": [0.001, 0.01]},
          ]

#          {"svc__C": [0.1, 1, 10, 100],
#           "svc__gamma": [0.00001,0.00005, 0.0001, 0.00015, 0.001, 0.01],
#           "contrastsubgraphtransformer__problem": [1, 2],
#           "contrastsubgraphtransformer__solver": [sdp, qp],
#           "contrastsubgraphtransformer__num_cs": [1, 2, 3],
#           "contrastsubgraphtransformer__percentile": [70, 80, 90],
#           "contrastsubgraphtransformer__percentile2": [70, 80, 90]}

pipe = make_pipeline(ContrastSubgraphTransformer(a_label="ASD", b_label="TD"),StandardScaler(), SVC())


inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Nested CV with parameter optimization
clf = GridSearchCV(estimator=pipe, param_grid=p_grid, cv=inner_cv)
nested_score = cross_val_score(clf, X=graphs, y=labels, cv=outer_cv)
nested_score.mean()

alpha = 0.036168132942326486 (70-th percentile), alpha2 = 0.04105571847507333 (70-th percentile)
Time for SDP: 0:00:05.590967
CS before local search: [10, 17, 20, 21, 29, 30, 31, 36, 37, 38, 39, 40, 41, 54, 55, 70, 71, 72, 73, 74, 75, 76, 77, 79, 83, 85, 89, 94, 95, 96, 97, 98, 99, 107, 112, 115]
Objective function value: 19.036168132942322
CS after local search: [ 10  17  20  21  28  29  30  31  36  37  38  39  40  41  54  55  70  71
  72  73  74  75  76  77  79  83  85  89  94  95  96  97  98  99 107 112
 115]
Objective function value: 19.05669599217987
Local Search improved solution by 0.10783608914455162%
Time for local search: 0:00:00.012689
Time to find CS: 0:00:05.603656
Time for SDP: 0:00:02.198840
CS before local search: [0, 4, 5, 6, 8, 9, 11, 12, 13, 15, 52, 53, 57, 58, 60, 61, 62, 69, 89, 90, 92, 98, 99, 100, 102, 103]
Objective function value: 7.756598240469204
CS after local search: [  4   5   6   8   9  11  12  13  15  52  53  57  58  60  61  62  69  89
  90  92 100 102 1

0.5247619047619049

In [ ]:
class DiscriminativeEdgesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, a_label: str, b_label: str, num_edges: int) -> None:
        self.a_label = a_label
        self.b_label = b_label

        self.num_edges = num_edges

        self.axes_labels = [f"% similarity between important {a_label} edges",
                            f"% similarity between important {b_label} edges",
                            f"% similarity of whole graph with {a_label} class"]

    def fit(self, graphs, labels) -> None:
        # Create and Write Summary Graphs
        summary_A = utils.summary_graph(graphs[np.where(labels == self.a_label)])
        summary_B = utils.summary_graph(graphs[np.where(labels == self.b_label)])
            
        # Get the difference network between the edge weights in group A and B
        # Note that (u,v) is the same as (v,u), so we extract the upper triangle of the matrix
        self.diff_net = np.triu(summary_A - summary_B, k=1)

        # Find the num_edges most positive and most negative edge diffs
        partitions = np.argpartition(self.diff_net, (self.num_edges, -self.num_edges), axis=None)
        top_n = np.unravel_index(partitions[-self.num_edges:], self.diff_net.shape)
        bottom_n = np.unravel_index(partitions[:self.num_edges], self.diff_net.shape)

        # Ensure the top edges are all positive and the bottom edges are all negative
        top_edges = self.diff_net[top_n]
        positive = top_edges > 0
        self.positive_indices = (top_n[0][positive], top_n[1][positive])
        self.important_a_edges = self.diff_net[self.positive_indices]

        bottom_edges = self.diff_net[bottom_n]
        negative = bottom_edges < 0
        self.negative_indices = (bottom_n[0][negative], bottom_n[1][negative])
        self.important_b_edges = self.diff_net[self.negative_indices]

        self.a_sum = np.sum(self.important_a_edges)
        self.b_sum = np.sum(self.important_b_edges)
        self.full_sum = np.sum(np.abs(self.diff_net))

    
    def transform(self, graph):
        points = np.array(list(map(self.graph_to_point, graphs)))
        return points

    def graph_to_point(self, graph):
        graph[np.where(graph==0)] = -1
        return np.array([100*np.dot(self.important_a_edges, graph[self.positive_indices])/self.a_sum,
                         100*np.dot(self.important_b_edges, graph[self.negative_indices])/self.b_sum,
                         100*np.sum(np.multiply(graph, self.diff_net))/self.full_sum])